In [1]:
import sys
import nltk
from numpy import *
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.rdd import RDD
#nltk.download('punkt')
#import nltk.data
#tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
#swedish_tokenizer = nltk.data.load('tokenizers/punkt/swedish.pickle')

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("local") \
        .appName("A.2")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [4]:
distFileEN = spark_context.textFile("europarl-v7.sv-en.en")
distFileSV = spark_context.textFile("europarl-v7.sv-en.sv")
#count the two files
#distFileEN.count()
#1862234
#distFileSV.count()
#1862234

In [5]:
TokenEN = distFileEN.pipe("/home/ubuntu/tools/tokenizer.perl -l en")
TokenSV = distFileSV.pipe("/home/ubuntu/tools/tokenizer.perl -l sv")
#TokenEN.cache()
#TokenSV.cache()
#SplitSenEN = TokenEN.pipe("/home/ubuntu/tools/split-sentences.perl -l en")
#SplitSenSV = TokenSV.pipe("/home/ubuntu/tools/split-sentences.perl -l sv")
#LowerCaseEN = TokenEN.map(lambda line: line.lower().split("/n"))
#LowerCaseSV = TokenSV.map(lambda line: line.lower().split("/n"))
LowerCaseEN = TokenEN.flatMap(lambda line: line.lower().split("/n"))
LowerCaseSV = TokenEN.flatMap(lambda line: line.lower().split("/n"))

In [ ]:
#count after pre-processing
#LowerCaseEN.count()
#1862234
LowerCaseSV.count()
#1862234

In [ ]:
LowerCaseSV.count()
#1862234

In [ ]:
TokenEN.take(20)

In [6]:
#A.3
SingleEN = LowerCaseEN.flatMap(lambda line: ''.join(line).split(" "))
SingleSV = LowerCaseSV.flatMap(lambda line: ''.join(line).split(" "))

In [ ]:
SingleEN.take(10)

In [7]:
#SingleEN.take(50)
#SingleSV.take(50)
EN_mapped = SingleEN.map(lambda x: (x,1))
EN_grouped = EN_mapped.groupByKey()

#SV_mapped = SingleSV.map(lambda x: (x,1))
#SV_grouped = SV_mapped.groupByKey()

In [ ]:
print(list((j[0], list(j[1])) for j in EN_grouped.take(5)))

In [8]:
EN_freq_of_words = EN_grouped.mapValues(sum).map(lambda x: (x[1],x[0]))\
.sortByKey(False) 
#SV_freq_of_words =SV_grouped.mapValues(sum).map(lambda x: (x[1],x[0])).sortByKey(False)


TypeError: unorderable types: ResultIterable() < ResultIterable()

In [ ]:
EN_freq_of_words.take(10)
#SV_freq_of_words.take(10)